In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 4000
pd.set_option('display.max_colwidth', -1)

In [2]:
sqlite_file = 'database.sqlite'    # name of the sqlite database file
# table names
# country
# league
# match
# player
# player_attributes
# team
# team_attributes

In [3]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [126]:
country = pd.read_sql_query("select * from country", conn)
league = pd.read_sql_query("select * from league", conn)
match = pd.read_sql_query("select * from match", conn)
player = pd.read_sql_query("select * from player", conn)
player_attributes = pd.read_sql_query("select * from player_attributes", conn)
team_attributes = pd.read_sql_query("select * from team_attributes", conn)
team = pd.read_sql_query("select * from team", conn)
player_position = pd.read_csv('./players_position.csv')

In [5]:
season_dict = {
    2007: {1: '2006/2007', 2: '2006/2007', 3: '2006/2007', 4: '2006/2007', 5: '2007/2008', 6: '2007/2008', 7: '2007/2008', 8: '2007/2008', 9: '2007/2008', 10: '2007/2008', 11: '2007/2008', 12: '2007/2008'},
    2008: {1: '2007/2008', 2: '2007/2008', 3: '2007/2008', 4: '2007/2008', 5: '2008/2009', 6: '2008/2009', 7: '2008/2009', 8: '2008/2009', 9: '2008/2009', 10: '2008/2009', 11: '2008/2009', 12: '2008/2009'},
    2009: {1: '2008/2009', 2: '2008/2009', 3: '2008/2009', 4: '2008/2009', 5: '2009/2010', 6: '2009/2010', 7: '2009/2010', 8: '2009/2010', 9: '2009/2010', 10: '2009/2010', 11: '2009/2010', 12: '2009/2010'},
    2010: {1: '2009/2010', 2: '2009/2010', 3: '2009/2010', 4: '2009/2010', 5: '2010/2011', 6: '2010/2011', 7: '2010/2011', 8: '2010/2011', 9: '2010/2011', 10: '2010/2011', 11: '2010/2011', 12: '2010/2011'},
    2011: {1: '2010/2011', 2: '2010/2011', 3: '2010/2011', 4: '2010/2011', 5: '2011/2012', 6: '2011/2012', 7: '2011/2012', 8: '2011/2012', 9: '2011/2012', 10: '2011/2012', 11: '2011/2012', 12: '2011/2012'},
    2012: {1: '2011/2012', 2: '2011/2012', 3: '2011/2012', 4: '2011/2012', 5: '2012/2013', 6: '2012/2013', 7: '2012/2013', 8: '2012/2013', 9: '2012/2013', 10: '2012/2013', 11: '2012/2013', 12: '2012/2013'},
    2013: {1: '2012/2013', 2: '2012/2013', 3: '2012/2013', 4: '2012/2013', 5: '2013/2014', 6: '2013/2014', 7: '2013/2014', 8: '2013/2014', 9: '2013/2014', 10: '2013/2014', 11: '2013/2014', 12: '2013/2014'},
    2014: {1: '2013/2014', 2: '2013/2014', 3: '2013/2014', 4: '2013/2014', 5: '2014/2015', 6: '2014/2015', 7: '2014/2015', 8: '2014/2015', 9: '2014/2015', 10: '2014/2015', 11: '2014/2015', 12: '2014/2015'},
    2015: {1: '2014/2015', 2: '2014/2015', 3: '2014/2015', 4: '2014/2015', 5: '2015/2016', 6: '2015/2016', 7: '2015/2016', 8: '2015/2016', 9: '2015/2016', 10: '2015/2016', 11: '2015/2016', 12: '2015/2016'},
    2016: {1: '2015/2016', 2: '2015/2016', 3: '2015/2016', 4: '2015/2016', 5: '2016/2017', 6: '2016/2017', 7: '2016/2017', 8: '2016/2017', 9: '2016/2017', 10: '2016/2017', 11: '2016/2017', 12: '2016/2017'}
}

In [6]:
team_attributes_2 = (
    team_attributes
    .merge(team[["team_api_id", "team_long_name", "team_short_name"]], on="team_api_id", how="left")
    .merge(match[["country_id", "home_team_api_id"]], left_on="team_api_id", right_on="home_team_api_id", how="left")
    .merge(league[["country_id", "name"]], on="country_id", how="left")
    .rename(columns={"name": "league_name"})
    .drop(["home_team_api_id", "country_id", "id", "team_fifa_api_id"], axis=1)
    .query("league_name == 'England Premier League'")
    .drop_duplicates()
    .reset_index(drop=True)
    .pipe(lambda x: x.assign(year=pd.to_datetime(x.date).dt.year))
    .pipe(lambda x: x.assign(month=pd.to_datetime(x.date).dt.month))
)
team_attributes_2['season'] = [season_dict[year][month] for (year, month) in zip(team_attributes_2.year, team_attributes_2.month)]

team_attributes_2.drop(['date', 'year', 'month'], axis=1, inplace=True)

In [7]:
team_attributes_2.head(2)

,team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,team_long_name,team_short_name,league_name,season
0,9825,66,Balanced,NaN,Little,30,Short,Free Form,30,Safe,45,Normal,35,Normal,Free Form,30,Deep,40,Press,50,Normal,Cover,Arsenal,ARS,England Premier League,2009/2010
1,9825,75,Fast,NaN,Little,40,Mixed,Free Form,40,Normal,45,Normal,65,Normal,Free Form,50,Medium,40,Press,45,Normal,Cover,Arsenal,ARS,England Premier League,2010/2011


In [72]:
player_attributes = (
    player_attributes
    .pipe(lambda x: x.assign(year=pd.to_datetime(x.date).dt.year))
    .pipe(lambda x: x.assign(month=pd.to_datetime(x.date).dt.month))
)
player_attributes['season'] = [season_dict[year][month] for (year, month) in zip(player_attributes.year, player_attributes.month)]

In [73]:
player_attributes_2 = (
    player_attributes
    .merge(player, on=['player_api_id', 'player_fifa_api_id'], how='left')
    .pipe(lambda x: x.assign(age=pd.to_datetime(x.date).dt.year - pd.to_datetime(x.birthday).dt.year))
    .drop(['player_name', 'birthday', 'id_x', 'id_y', 'player_fifa_api_id'], axis=1)
)

In [74]:
selected_columns = [col for col in player_attributes_2.columns if col not in ('id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'year', 'month', 'season')]

In [75]:
player_attributes_2 = (
    player_attributes_2
    .groupby(["player_api_id", "season"])
    .agg(dict(zip(selected_columns, ["mean"]*len(selected_columns))))
    .reset_index()
)

In [76]:
player_attributes_2.head(2)

,player_api_id,season,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,height,weight,age
0,2625,2006/2007,63.0,64.0,48.0,48.0,47.0,64.0,38.0,57.0,50.0,46.0,67.0,57.0,67.0,64.0,59.0,52.0,49.0,61.0,56.0,78.0,56.0,59.0,72.0,52.0,55.0,56.0,46.0,64.0,66.0,63.0,14.0,11.0,67.0,9.0,10.0,175.26,154.0,26.0
1,2625,2007/2008,63.0,64.0,48.0,48.0,47.0,64.0,38.0,57.0,50.0,51.0,67.0,57.0,67.0,64.0,59.0,52.0,49.0,61.0,56.0,78.0,56.0,59.0,72.0,52.0,55.0,56.0,46.0,64.0,66.0,63.0,14.0,24.0,67.0,24.0,24.0,175.26,154.0,26.0


In [13]:
match_selected_columns=[
    'season', 'match_api_id', 'home_team_api_id', 'away_team_api_id',
    'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 
    'home_player_X9', 'home_player_X10', 'home_player_X11',
    'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 
    'away_player_X9', 'away_player_X10', 'away_player_X11',
    'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 
    'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
    'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 
    'away_player_Y9', 'away_player_Y10', 'away_player_Y11',
    'home_player_1','home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 
    'home_player_9', 'home_player_10', 'home_player_11',
    'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 
    'away_player_9', 'away_player_10', 'away_player_11'
]

In [14]:
epl_matches = (
    match
    .query("country_id==1729")
    [match_selected_columns]
    .fillna(0)
)

In [15]:
epl_matches.head(2)

,season,match_api_id,home_team_api_id,away_team_api_id,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
1728,2008/2009,489042,10260,10261,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,30726.0,30362.0,30620.0,30865.0,32569.0,24148.0,34944.0,30373.0,24154.0,24157.0,30829.0,24224.0,25518.0,24228.0,30929.0,29581.0,38807.0,40565.0,30360.0,33852.0,34574.0,37799.0
1729,2008/2009,489043,9825,8659,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,5.0,7.0,9.0,1.0,3.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,23686.0,26111.0,38835.0,30986.0,31291.0,31013.0,30935.0,39297.0,26181.0,30960.0,36410.0,36373.0,36832.0,23115.0,37280.0,24728.0,24664.0,31088.0,23257.0,24171.0,25922.0,27267.0


In [46]:
home_player_cols= ['home_player_1','home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 
                    'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11']
away_player_cols = ['away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 
                    'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']

In [92]:
def get_team(df, min_max='min', home_away_id='home_team_api_id'):
    final_df = (
        df
        [['season', '{}'.format(col), '{}'.format(home_away_id)]]
        .drop_duplicates()
        .groupby(["season", "{}".format(col)])
        .agg({"{}".format(home_away_id): "min"})
        .reset_index()
        .rename(columns={"{}".format(col): "player_api_id", "{}".format(home_away_id): "team_api_id"})
    )
    return final_df

In [102]:
first = True
for col in home_player_cols:
    if first is True:
        first_df = get_team(epl_matches, min_max='min', home_away_id='home_team_api_id')
        second_df = get_team(epl_matches, min_max='max', home_away_id='home_team_api_id')
        home_final = pd.concat([first_team, second_team], axis=0)
        first = False
    else:
        first_team = get_team(epl_matches, min_max='min', home_away_id='home_team_api_id')
        second_team = get_team(epl_matches, min_max='max', home_away_id='home_team_api_id')
        home_intermediate = pd.concat([first_team, second_team], axis=0)
        home_final = pd.concat([home_intermediate, home_final], axis=0)
        
home_final = (
    home_final
    .drop_duplicates()
    .query("player_api_id != 0")
    .reset_index(drop=True)
)

In [104]:
first = True
for col in away_player_cols:
    if first is True:
        first_df = get_team(epl_matches, min_max='min', home_away_id='away_team_api_id')
        second_df = get_team(epl_matches, min_max='max', home_away_id='away_team_api_id')
        away_final = pd.concat([first_team, second_team], axis=0)
        first = False
    else:
        first_team = get_team(epl_matches, min_max='min', home_away_id='away_team_api_id')
        second_team = get_team(epl_matches, min_max='max', home_away_id='away_team_api_id')
        away_intermediate = pd.concat([first_team, second_team], axis=0)
        away_final = pd.concat([away_intermediate, away_final], axis=0)
        
away_final = (
    away_final
    .drop_duplicates()
    .query("player_api_id != 0")
    .reset_index(drop=True)
)

In [111]:
player_team = pd.concat([home_final, away_final], axis=0)

In [112]:
player_team = (
    player_team
    .drop_duplicates()
    .reset_index(drop=True)
)

In [115]:
player_team.query("player_api_id == 38836 and season == '2008/2009'")

,season,player_api_id,team_api_id
2287,2008/2009,38836.0,8472
3519,2008/2009,38836.0,8586


In [124]:
player_attributes_3 = (
    player_attributes_2
    .merge(player_team, on=['season', 'player_api_id'], how='left')
    .merge(team_attributes_2[['team_api_id', 'season', 'league_name']], on=['team_api_id', 'season'], how='left')
    .query("league_name == 'England Premier League'")
    [['league_name', 'player_api_id', 'season', 'team_api_id', 'age', 'height', 'weight', 'overall_rating', 'potential', 
      'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control',
      'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 
      'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 
      'gk_diving','gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']]
)

In [125]:
player_attributes_3.head(2)

,league_name,player_api_id,season,team_api_id,age,height,weight,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
53,England Premier League,2802,2013/2014,10003,28.571429,172.72,159.0,76.0,76.857143,76.285714,71.0,53.0,68.857143,65.0,85.0,83.0,77.0,69.0,81.0,80.142857,77.571429,84.0,69.0,80.714286,76.0,67.0,59.0,51.0,77.0,54.0,49.0,73.0,73.142857,72.0,42.0,47.0,36.0,13.0,16.0,9.0,9.0,7.0
183,England Premier League,3520,2009/2010,9825,28.500000,172.72,154.0,85.0,90.500000,85.000000,88.0,50.0,90.000000,77.0,93.0,77.0,76.0,80.0,91.0,88.000000,87.000000,86.0,85.0,75.000000,84.0,58.0,76.0,67.0,85.0,49.0,75.0,86.0,85.000000,88.0,22.0,29.0,28.0,7.0,24.0,80.0,24.0,24.0


In [130]:
player_position.columns

Index(['playerID', 'CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LF', 'LM',
       'LW', 'LWB', 'RB', 'RF', 'RM', 'RW', 'RWB', 'ST', 'SW'],
      dtype='object')

In [131]:
player_position.query("playerID == 2802")

,playerID,CAM,CB,CDM,CF,CM,GK,LB,LF,LM,LW,LWB,RB,RF,RM,RW,RWB,ST,SW


In [132]:
player.head(2)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146


In [16]:
country_league = (
    league
    .merge(country, left_on='country_id', right_on='id', how='left')
    .rename(columns={"name_x": "league_name", "name_y": "country"})
    [["country_id", "country", "league_name"]]
)

In [37]:
columns=[
    'id', 'country_id', 'country', 'league_name', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'home_team_long_name', 'home_team_short_name', 
    'away_team_api_id', 'away_team_long_name', 'away_team_short_name', 'home_team_goal', 'away_team_goal',
    'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11',
    'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11',
    'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
    'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11',
    'home_player_1','home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
    'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11',
]#'goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']

In [38]:
match_country_league = (
    match
    .merge(country_league, on='country_id', how='left')
    .merge(team[["team_api_id", "team_long_name", "team_short_name"]], left_on="home_team_api_id", right_on="team_api_id", how="left")
    .rename(columns={"team_long_name": "home_team_long_name", "team_short_name": "home_team_short_name"})
    .drop("team_api_id", axis=1)
    .merge(team[["team_api_id", "team_long_name", "team_short_name"]], left_on="away_team_api_id", right_on="team_api_id", how="left")
    .rename(columns={"team_long_name": "away_team_long_name", "team_short_name": "away_team_short_name"})
    .drop("team_api_id", axis=1)
    [columns]
    .query("league_name == 'England Premier League'")
)

In [ ]:
match[match.home_player_1.notnull()]

In [39]:
match_country_league.head()

,id,country_id,country,league_name,season,stage,date,match_api_id,home_team_api_id,home_team_long_name,home_team_short_name,away_team_api_id,away_team_long_name,away_team_short_name,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
1728,1729,1729,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,489042,10260,Manchester United,MUN,10261,Newcastle United,NEW,1,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,30726.0,30362.0,30620.0,30865.0,32569.0,24148.0,34944.0,30373.0,24154.0,24157.0,30829.0,24224.0,25518.0,24228.0,30929.0,29581.0,38807.0,40565.0,30360.0,33852.0,34574.0,37799.0
1729,1730,1729,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,489043,9825,Arsenal,ARS,8659,West Bromwich Albion,WBA,1,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,5.0,7.0,9.0,1.0,3.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,23686.0,26111.0,38835.0,30986.0,31291.0,31013.0,30935.0,39297.0,26181.0,30960.0,36410.0,36373.0,36832.0,23115.0,37280.0,24728.0,24664.0,31088.0,23257.0,24171.0,25922.0,27267.0
1730,1731,1729,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,489044,8472,Sunderland,SUN,8650,Liverpool,LIV,0,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,32562.0,38836.0,24446.0,24408.0,36786.0,38802.0,24655.0,17866.0,30352.0,23927.0,24410.0,30660.0,37442.0,30617.0,24134.0,414792.0,37139.0,30618.0,40701.0,24800.0,24635.0,30853.0
1731,1732,1729,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,489045,8654,West Ham United,WHU,8528,Wigan Athletic,WIG,2,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,6.0,8.0,4.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,36374.0,30966.0,23818.0,37277.0,30687.0,36394.0,37169.0,24223.0,24773.0,34543.0,23139.0,34421.0,34987.0,35472.0,111865.0,25005.0,35327.0,25150.0,97988.0,41877.0,127857.0,34466.0
1732,1733,1729,England,England Premier League,2008/2009,1,2008-08-17 00:00:00,489046,10252,Aston Villa,AVL,8456,Manchester City,MCI,4,2,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,30380.0,30357.0,24658.0,43280.0,23282.0,38609.0,24780.0,23782.0,23354.0,23264.0,26165.0,31432.0,46403.0,24208.0,23939.0,33963.0,47413.0,40198.0,42119.0,NaN,33633.0,107216.0
